# Cleaning




Columns are the same for each dataset.     
So we can write one script to clean them all. (Test on one for loop on others)

### Imports and setup

In [226]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import sys
import demoji
import nltk 
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')


sys.path.append('../')



pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)



data_path = os.path.join('combined_files.csv')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paganinik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Before data can be read in to dataframe, I think it is necessary to do some preprocessing to the csv file itself

### Preprocessing has been done in clean.py


## Still some more data cleaning needs to be done

In [227]:
df = pd.read_csv(data_path)

In [228]:
df.head(1)

,created,id,author,retrieved,edited,pinned,archived,locked,removed,deleted,is_self,is_video,is_original_content,title,link_flair_text,upvote_ratio,score,gilded,total_awards_received,num_comments,num_crossposts,selftext,thumbnail,shortlink
0,2021-01-01 00:02:06,ko124i,[deleted],2021-02-02 21:52:13,1970-01-01 00:00:00,0,0,0,1,1,1,0,0,3k - 170k since March (Also buy LIT!!),Gain,1.0,34,0,1,14,0,[deleted],default,https://redd.it/ko124i


In [229]:
df.columns[17:20]

Index(['gilded', 'total_awards_received', 'num_comments'], dtype='object')

### Converting types

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1483575 entries, 0 to 1483574
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   created                1483575 non-null  object 
 1   id                     1483575 non-null  object 
 2   author                 1483575 non-null  object 
 3   retrieved              1483575 non-null  object 
 4   edited                 1483575 non-null  object 
 5   pinned                 1483575 non-null  int64  
 6   archived               1483575 non-null  int64  
 7   locked                 1483575 non-null  int64  
 8   removed                1483575 non-null  int64  
 9   deleted                1483575 non-null  int64  
 10  is_self                1483575 non-null  int64  
 11  is_video               1483575 non-null  int64  
 12  is_original_content    1483575 non-null  int64  
 13  title                  1483573 non-null  object 
 14  link_flair_text   

In [231]:


# date time columns
df['created'] =  pd.to_datetime(df['created'], format='%Y-%m-%d %H:%M:%S.%f')
df['retrieved'] =  pd.to_datetime(df['retrieved'], format='%Y-%m-%d %H:%M:%S.%f')
df['edited'] =  pd.to_datetime(df['edited'], format='%Y-%m-%d %H:%M:%S.%f')

# boolean / categorical variables
df['pinned'] = df['pinned'].astype('bool')
df['archived'] = df['archived'].astype('bool')
df['locked'] = df['locked'].astype('bool')
df['removed'] = df['removed'].astype('bool')
df['deleted'] = df['deleted'].astype('bool')
df['is_self'] = df['is_self'].astype('bool')
df['is_video'] = df['is_video'].astype('bool')
df['is_original_content'] = df['is_original_content'].astype('bool')

# int types
df['score'] = df['score'].astype('int')
df['gilded'] = df['gilded'].astype('int')
df['total_awards_received'] = df['total_awards_received'].astype('int')
df['num_comments'] = df['num_comments'].astype('int')
df['num_crossposts'] = df['num_crossposts'].astype('int')



Columns:    
| Index | Feature               | Type     | Description                                                    | 
|-------|-----------------------|----------|----------------------------------------------------------------|
| 0     | id                    | string   | The id of the submission                                       |
| 1     | author                | string   | The redditors username                                         |
| 2     | created               | datetime | Time the submission was created                                |
| 3     | retrieved             | datetime | Time the submission was retrieved                              |
| 4     | edited                | datetime | Time the submission was modified                               |
| 5     | pinned                | boolean  | Whether or not the submission is pinned                        |
| 6     | archived              | boolean  | Whether or not the submission is archived                      |
| 7     | locked                | boolean  | Whether or not the submission is locked                        |
| 8     | removed               | boolean  | Whether or not the submission is removed                       |
| 9     | deleted               | boolean  | Whether or not the submission is user deleted                  |
| 10    | is_self               | boolean  | Whether or not the submission is a text                        |
| 11    | is_video              | boolean  | Whether or not the submission is a video                       |
| 12    | is_original_content   | boolean  | Whether or not the submission has been set as original content |
| 13    | title                 | string   | Title of the submission                                        |
| 14    | link_flair_text       | string   | Submission link flairs text content                            |
| 15    | upvote_ratio          | double   | Percentage of upvotes from all votes on submission             |
| 16    | score                 | integer  | number of upvotes                                              |
| 17    | gilded                | integer  | number of gilded awards                                        |
| 18    | total_awards_received | integer  | number of awards on the submission                             |
| 19    | num_comments          | integer  | number of comments on the submission                           |
| 20    | num_crossposts        | integer  | number of crossposts on the submission                         |
| 21    | selftext              | string   | submission selftext on text posts                              |
| 22    | thumbnail             | string   | submission thumbnail on image posts                            |
| 23    | shortlink             | string   | submission short url                                           |    

### Cleaning functions for Title


Things that need to be cleaned from "Title":
- New lines
- Emojis (convert or remove?)
- Spam messages (possibly only take posts that have a certain number of upvotes)
- links (need to remove entire record if link is only thing)
- videos (same as link)
- A lot of records do not talk about a specific stock. (Remove them?)

In [232]:
for col in df.columns[5:11]:
    print(df[col].value_counts())

False    1483575
Name: pinned, dtype: int64
False    1483575
Name: archived, dtype: int64
False    1478803
True        4772
Name: locked, dtype: int64
True     997962
False    485613
Name: removed, dtype: int64
False    1345959
True      137616
Name: deleted, dtype: int64
True     945261
False    538314
Name: is_self, dtype: int64


Columns that can be removed for sure:
- id
- shortlink  
- thumbnail
- retrieved
- edited 
- pinned  
- archived  
- locked  
- removed (if removed is true should we discard the record?)
- deleted (same as removed)
- is_self   
- is_video (use as flag to remove records?)
- gilded   

Maybe keep:   (general stats about the post)
- score
- upvote_ratio
- comments

Keep:     
- created
- title + selftext




Can we combine comments into a score. The score could be a weighted average of upvote ratio, comments, etc.

In [233]:
df['selftext'].value_counts()[0:4]

[removed]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

A lot of self-text posts remove the text after or delete it. And there are some additional messages that seem to be from bots that we could probably remove as well. I suggest removing 'deleted' and 'removed' and have an empty string instead. I think we should also combine this column with the title column so we only have one column with text. 

In [234]:
df[(df['is_self'] == 1) & (df['removed'] == 1)]

,created,id,author,retrieved,edited,pinned,archived,locked,removed,deleted,is_self,is_video,is_original_content,title,link_flair_text,upvote_ratio,score,gilded,total_awards_received,num_comments,num_crossposts,selftext,thumbnail,shortlink
0,2021-01-01 00:02:06,ko124i,[deleted],2021-02-02 21:52:13,1970-01-01,False,False,False,True,True,True,False,False,3k - 170k since March (Also buy LIT!!),Gain,1.00,34,0,1,14,0,[deleted],default,https://redd.it/ko124i
9,2021-01-01 00:13:41,ko190a,[deleted],2021-02-03 21:12:56,1970-01-01,False,False,False,True,True,True,False,False,TSXV ROVR OTCQB ROVMF could be getting ready t...,General Discussion,1.00,1,0,0,0,0,[deleted],default,https://redd.it/ko190a
11,2021-01-01 00:18:03,ko1bnp,dluther93,2021-02-02 21:52:13,1970-01-01,False,False,False,True,False,True,False,False,What would make GME shorts win?,Discussion,1.00,1,0,0,0,0,[removed],default,https://redd.it/ko1bnp
15,2021-01-01 00:18:57,ko1c6o,[deleted],2021-02-03 21:17:46,1970-01-01,False,False,False,True,True,True,False,False,Stocks for beginners: How do you know which st...,None,0.55,1,0,0,14,0,[deleted],default,https://redd.it/ko1c6o
16,2021-01-01 00:18:57,ko1c6o,[deleted],2021-02-03 21:17:46,1970-01-01,False,False,False,True,True,True,False,False,Stocks for beginners: How do you know which st...,None,0.55,1,0,0,14,0,[deleted],default,https://redd.it/ko1c6o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483564,2021-12-31 23:46:45,rt6fuz,[deleted],2022-01-01 03:29:07,1970-01-01,False,False,False,True,False,True,False,False,I'm about to start making $120K salary. Should...,Auto,1.00,1,0,0,1,0,[removed],default,https://redd.it/rt6fuz
1483565,2021-12-31 23:48:27,rt6gxc,peachezandsteam,2022-01-01 03:29:52,1970-01-01,False,False,False,True,False,True,False,False,Is the notable congresswoman's GOOG calls a gu...,Discussion,0.66,1,0,0,0,0,[removed],https://b.thumbs.redditmedia.com/pp0YjoMYmhccq...,https://redd.it/rt6gxc
1483571,2021-12-31 23:55:49,rt6lul,coyote_of_the_month,2022-01-01 03:29:07,1970-01-01,False,False,False,True,False,True,False,False,Company was unable to process additional 403(b...,R5: Legal,1.00,1,0,0,3,0,[removed],self,https://redd.it/rt6lul
1483572,2021-12-31 23:55:51,rt6lv6,[deleted],2022-01-01 03:56:58,1970-01-01,False,False,False,True,False,True,False,False,Winner or loser? Only time will tell. 2021 end...,Discussion,1.00,1,0,0,1,0,[removed],default,https://redd.it/rt6lv6


In [235]:
df['thumbnail'].value_counts()

default                                                                             849993
self                                                                                431161
image                                                                               113541
nsfw                                                                                  3317
spoiler                                                                               1925
                                                                                     ...  
https://a.thumbs.redditmedia.com/m8QV6nOfMondOzYlaoRpWQ2qjYjz0SB5DezQixtPfM8.jpg         1
https://a.thumbs.redditmedia.com/UvYbtr6hGZ3WFrk32-3oT-hJ18H50aStwX8RF08zKn8.jpg         1
https://a.thumbs.redditmedia.com/wL3ECjUia5J3zuQ3dVSnqtdMUag0o2sie4tMQjXXRu8.jpg         1
https://b.thumbs.redditmedia.com/3b2alO76OukxzyuDfWZzxld-1X7UqPsuvjqslCtvfwU.jpg         1
https://b.thumbs.redditmedia.com/TtUVXN1XpoXXuzY85bJMNo1451L4fTOqYqKailX9M-c.jpg         1

In [236]:
df.columns

Index(['created', 'id', 'author', 'retrieved', 'edited', 'pinned', 'archived',
       'locked', 'removed', 'deleted', 'is_self', 'is_video',
       'is_original_content', 'title', 'link_flair_text', 'upvote_ratio',
       'score', 'gilded', 'total_awards_received', 'num_comments',
       'num_crossposts', 'selftext', 'thumbnail', 'shortlink'],
      dtype='object')

### We just want the text for the most part

Experimenting with upvote_ratio and score as well.


In [237]:
df = df[['created','removed', 'deleted', 'is_self','title', 'upvote_ratio', 'score', 'selftext']]

pseudocode:   
```
if removed or deleted:    
    just take title  and date

if is_self and not removed or deleted and type(selftext) is string:
    take date, title + selftext
else:
    take date, title

```

In [238]:
df_extracted = df.loc[df['is_self'] & ~(df['removed'] | df['deleted']) & (df['selftext'].apply(lambda x: type(x) == str))]
df_extracted['text'] = df_extracted['title'] + ' ' + df_extracted['selftext']
df_extracted = df_extracted[['created', 'text', 'upvote_ratio', 'score']]

C:\Users\paganinik\AppData\Local\Temp\ipykernel_2184\1633618752.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['text'] = df_extracted['title'] + ' ' + df_extracted['selftext']


In [239]:
df_extracted.head()

,created,text,upvote_ratio,score
3,2021-01-01 00:05:17,Advice for someone who's never dealt with stoc...,0.40,0
4,2021-01-01 00:05:17,Advice for someone who's never dealt with stoc...,0.40,0
7,2021-01-01 00:13:13,So /r/stocks what was your 2020 investment rat...,0.63,4
8,2021-01-01 00:13:13,So /r/stocks what was your 2020 investment rat...,0.63,4
10,2021-01-01 00:15:38,WSBVoteBot Log for Jan 01 2021 Every time a ne...,0.50,0


In [240]:
df_extracted.shape

(306368, 4)

In [241]:

df_extracted['text'].head(20)


3     Advice for someone who's never dealt with stoc...
4     Advice for someone who's never dealt with stoc...
7     So /r/stocks what was your 2020 investment rat...
8     So /r/stocks what was your 2020 investment rat...
10    WSBVoteBot Log for Jan 01 2021 Every time a ne...
14    Hedging your portfolio Just out of curiosity  ...
20    BNGO Bear Case (Serious) I'm actually quite sk...
21    Daily Executions- December 31 2020 Hi Everyone...
35    Built two Google Sheets templates with automat...
36    Built two Google Sheets templates with automat...
41    $GAXY Youtuber London Investor will interview ...
47    Thoughts on Old School Value's stock tracking ...
57    GME is the Rockets 🚀🚀🚀🚀 Gamestop colors: Red  ...
58    Western Digital (WDC) rose 11.83% today. Anybo...
59    ARK invest selling $TSLA Ark invest ETF’s ARKW...
60    ARK invest selling $TSLA Ark invest ETF’s ARKW...
66    Recent IPO Chindata (CD). Looks promising. Wha...
67    Recent IPO Chindata (CD). Looks promising.

### Removing emojis

In [242]:
df_extracted['text'] = df_extracted['text'].str.encode('ascii', 'ignore').str.decode('ascii')

### Removing links

In [243]:
df_extracted['text'][10]

'WSBVoteBot Log for Jan 01 2021 Every time a new submission is posted to wallstreetbets /u/wsbvotebot posts a comment that allows you to click and vote to remove that submission. This is the log of volunteer moderators actions which you can vote to reverse. [Check the leaderboard](https://www.reddit.com/r/wallstreetbets/wiki/leaderboard) to see who is doing the most to keep /r/wallstreetbets great.User commentary as replies to the messages below are encouraged. Report bugs to /u/zjz.'

In [244]:
import re

#regex from chatgpt seems to work
url_pattern = re.compile(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})")
df_extracted['text'] = df_extracted['text'].str.replace(url_pattern,'')

### Remove reddit mentions (Maybe)

In [245]:
df_extracted['text'] = df_extracted['text'].str.replace(re.compile(r"(\/u\/[a-zA-Z0-9]+|\/r\/[a-zA-Z0-9]+)"),'')

### Remove Duplicate texts (likely from bots)

In [246]:
before = df_extracted.shape[0]
print(f'Shape before drop duplicates: {df_extracted.shape}')
df_extracted = df_extracted.drop_duplicates(subset=["text"], keep=False)
print(f'Shape after drop duplicates: {df_extracted.shape}')
print(f'Records lost: {before - df_extracted.shape[0]}')

Shape before drop duplicates: (306368, 4)
Shape after drop duplicates: (302003, 4)
Records lost: 4365


In [247]:
df_extracted['text']

10         WSBVoteBot Log for Jan 01 2021 Every time a ne...
14         Hedging your portfolio Just out of curiosity  ...
20         BNGO Bear Case (Serious) I'm actually quite sk...
21         Daily Executions- December 31 2020 Hi Everyone...
41         $GAXY Youtuber London Investor will interview ...
                                 ...                        
1483539    Why do people open up multiple positions of th...
1483546    Five penny stocks to put on your watchlist in ...
1483548    Any suggestion on what to do with an employer ...
1483566    Best Keeper credit cards? What are the best ke...
1483570    im a teen and i want to start investing in ind...
Name: text, Length: 302003, dtype: object

### Extracting what each post is about (ticker information)


Need to add more tickers

In [248]:
import json

ticker_dict = 0

with open('stonks.json', 'r') as f:
    ticker_dict = json.load(f)
    


def find_ticker(text):
    mentioned = []
    for ticker, names in ticker_dict.items():
        for name in names:
            if name in text:
                mentioned.append(ticker)
                break
    return " ".join(mentioned)

df_extracted['mentioned'] = df_extracted['text'].apply(find_ticker)


In [249]:
df_extracted['mentioned'].value_counts()

             277195
MSFT          20203
TSLA           3562
MSFT TSLA      1043
Name: mentioned, dtype: int64

In [250]:
df_extracted['mentioned'].info()

<class 'pandas.core.series.Series'>
Int64Index: 302003 entries, 10 to 1483570
Series name: mentioned
Non-Null Count   Dtype 
--------------   ----- 
302003 non-null  object
dtypes: object(1)
memory usage: 4.6+ MB


In [251]:
df_extracted.to_csv('just_dates_and_text.csv', index=False)

In [252]:
df_extracted.head()

,created,text,upvote_ratio,score,mentioned
10,2021-01-01 00:15:38,WSBVoteBot Log for Jan 01 2021 Every time a ne...,0.50,0,
14,2021-01-01 00:18:40,Hedging your portfolio Just out of curiosity ...,0.60,2,
20,2021-01-01 00:24:04,BNGO Bear Case (Serious) I'm actually quite sk...,0.74,42,
21,2021-01-01 00:24:14,Daily Executions- December 31 2020 Hi Everyone...,0.91,9,
41,2021-01-01 00:42:36,$GAXY Youtuber London Investor will interview ...,0.97,31,


### Removing punctuation

In [253]:
df_extracted['text'] = df_extracted['text'].replace('[^\w\s]', '', regex=True)


### Removing capitalization

In [254]:
df_extracted['text'] = df_extracted['text'].str.lower()



### Stop word removal????

In [255]:
stop_word_list = set(stopwords.words('english'))


df_extracted['text'] = df_extracted['text'].map(lambda x : " ".join(w for w in x.split() if w not in stop_word_list))

### Stemming?

The following cell takes a while to run (under 5 min)

In [256]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize




nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()
df_extracted['text'] = df_extracted['text'].apply(lambda x: " ".join(lemmatizer.lemmatize(word) for word in word_tokenize(x)))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\paganinik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paganinik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [257]:
df_extracted['text']

10         wsbvotebot log jan 01 2021 every time new subm...
14         hedging portfolio curiosity anyone make move h...
20         bngo bear case serious im actually quite skept...
21         daily execution december 31 2020 hi everyone i...
41         gaxy youtuber london investor interview coo ga...
                                 ...                        
1483539    people open multiple position pair whats benef...
1483546    five penny stock put watchlist 2022 llnw limel...
1483548    suggestion employer pay hi young man working s...
1483566    best keeper credit card best keeper card know ...
1483570    im teen want start investing index stock sp500...
Name: text, Length: 302003, dtype: object

How to do the word embedding thing


https://www.youtube.com/watch?v=ZogxNcyqVqE&ab_channel=TheAIUniversity


https://www.guru99.com/word-embedding-word2vec.html

### Word Embeddings - Word2Vec




Training the word2vec model

### Figuring out which companies we can predict for

Since, it would be unfair to ask a model to predict the price of a stock that is not mentioned in the data that is given, we need to do something about it.   

If we are asking the model to predict the price for Tesla in one hour based off the reddit comments from the previous 5 hours, Tesla would need to be mentioned in the previous 5 hours.     

I think a threshold of maybe like at least 5 posts in the last 5 hours to be included as a training example.     

To decrease the likelihood of the the target company not being mentioned we can increase the time window 